In [4]:
import sys
sys.path.append('../src')
from data_loader import data_loader
from GCN_model import n_hidden_GCN
# from data import get_data

In [3]:
loader = data_loader("../data/voting_features.csv", "../data/edges.csv")
features, labels, A = loader.get_data()

In [6]:
model = n_hidden_GCN(A, features, labels, hidden_neurons=200)
model.train_epoch(epochs=1, lr=1e-3)

../src/GCN_model.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)


AssertionError: Torch not compiled with CUDA enabled

In [9]:
import pandas as pd
edges = pd.read_csv("../data/raw/handles.csv")
edges

,Name,Handle,Party
0,AK Lisa Murkowski,lisamurkowski,R
1,AK Dan Sullivan,SenDanSullivan,R
2,AL Doug Jones,SenDougJones,D
3,AL Richard C. Shelby,SenShelby,R
4,AR John Boozman,JohnBoozman,R
...,...,...,...
95,WI Ron Johnson,SenRonJohnson,R
96,WV Shelley Moore Capito,SenCapito,R
97,WV Joe Manchin III,Sen_JoeManchin,D
98,WY John Barrasso,SenJohnBarrasso,R


In [6]:
rows, cols = edges[0], edges[1]

In [7]:
rows

0                      followed
1                UT Mitt Romney
2           CA Kamala D. Harris
3           MD Chris Van Hollen
4       NH Margaret Wood Hassan
                 ...           
3817        ME Susan M. Collins
3818       IL Richard J. Durbin
3819               OR Ron Wyden
3820      NY Charles E. Schumer
3821      CT Christopher Murphy
Name: 0, Length: 3822, dtype: object

In [76]:
import numpy as np
import networkx as nx
import pandas as pd
import torch
class data_loader():
    def __init__(self, feature_address, edges_address, directed = False):
        features = pd.read_csv(feature_address)
        print(features)
        edges = pd.read_csv(edges_address)

        #adjacency matrix
        adj = self.get_adj(edges, directed)    
    
        self.labels = torch.LongTensor(features['79'])    
        self.features = np.array(features.iloc[:, 1:features.shape[1]-1])
        self.features = torch.FloatTensor(self.features)

        self.A = torch.from_numpy(adj).float()
    def get_adj(self, edges, directed):
        rows, cols = edges["followed"], edges["following"]
    
        nodes = list(set(rows).union(set(cols)))
        n_nodes = len(nodes)
    
        node_index = {}
        for i in np.arange(len(nodes)):
            node_index[nodes[i]] = i
            i += 1
    
        adj = np.zeros((n_nodes, n_nodes), dtype='float32')

        for i in range(len(edges)):
            adj[node_index[rows[i]], node_index[cols[i]]]  = 1.0
            if not directed: 
                adj[node_index[cols[i]], node_index[rows[i]]]  = 1.0 
            
        return adj
    
    def get_data(self):
        return self.features, self.labels, self.A


In [78]:
loader = data_loader("../data/voting_features.csv", "../data/edges.csv")

               Senator Name  0  1  2  3  4  5  6  7  8  ...  70  71  72  73  \
0           AK Dan Sullivan  0  0  0  0  2  2  2  2  0  ...   0   0   0   0   
1         AK Lisa Murkowski  0  0  0  0  2  2  0  2  0  ...   0   0   0   0   
2             AL Doug Jones  0  0  0  0  0  0  0  2  0  ...   0   2   2   2   
3      AL Richard C. Shelby  0  0  0  0  2  2  2  2  0  ...   0   0   0   0   
4           AR John Boozman  0  0  0  0  2  2  2  2  0  ...   0   0   0   0   
..                      ... .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..   
95         WI Tammy Baldwin  0  0  0  0  0  0  0  0  2  ...   2   2   2   2   
96       WV Joe Manchin III  0  0  0  0  0  0  0  2  0  ...   0   2   2   2   
97  WV Shelley Moore Capito  0  0  0  0  2  2  2  2  0  ...   0   0   0   0   
98         WY John Barrasso  0  0  0  0  2  2  2  2  0  ...   0   0   0   0   
99       WY Michael B. Enzi  2  2  2  0  2  2  2  2  0  ...   0   0   0   0   

    74  75  76  77  78  79  
0    0   0   0   0   0

In [85]:
features, labels, A = loader.get_data()
len(A)

100

In [61]:
voting = pd.read_csv("../data/voting_data.csv")

In [65]:
voting["Decision"] = voting[""]

,Senator Name,Decision,Bill
0,TN Lamar Alexander,Yea,0
1,WI Tammy Baldwin,Yea,0
2,WY John Barrasso,Yea,0
3,CO Michael F. Bennet,Yea,0
4,TN Marsha Blackburn,Yea,0
...,...,...,...
7895,MA Elizabeth Warren,Nay,78
7896,RI Sheldon Whitehouse,Nay,78
7897,MS Roger F. Wicker,Yea,78
7898,OR Ron Wyden,Nay,78


VA Tim Kaine            79
WY John Barrasso        79
MI Debbie Stabenow      79
MN Tina Smith           79
CA Dianne Feinstein     79
                        ..
ND Kevin Cramer         79
AL Richard C. Shelby    79
LA Bill Cassidy         79
TX Ted Cruz             79
ME Susan M. Collins     79
Name: Senator Name, Length: 100, dtype: int64